In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.append('../src/')

In [2]:
APPLIANCE = ['fridge', 'dish_washer', 'washing_machine']
THRESHOLD = [50., 10., 20.]
MIN_ON = [1., 30., 30.]
MIN_OFF = [1., 3., 30.]

METER = 'aggregate'
SEQ_LEN = 60*8
BORDER = 16
BATCH_SIZE = 32

MAX_POWER = 2000

### UKDale data load

In [3]:
from utils.preprocessing import get_status
houses=[1,2,5]
ds_meter = []
ds_appliance = []
ds_status = []
for i in houses:
    ds = pd.read_feather('../data/ukdale/feather_files/UKDALE_%d_train.feather' %(i))
    ds.set_index('datetime', inplace=True)
  
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    ds_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(ds_meter[i]) for i in range(3)]

### CHAIN2GATE filtering

In [4]:
from utils.preprocessing import chain2filter
houses=[1,2,5]
THRESHOLDS=[50,100,150,200,250,300]
datafilter=[]
for threshold in THRESHOLDS:
    original = []
    for k in houses:
        ds = pd.read_feather('../data/ukdale/feather_files/UKDALE_%d_train.feather' %(k))
        ds.set_index('datetime', inplace=True)  
        meter = ds[METER]
        original.append(meter)
    datafilter.append(chain2filter(original, threshold, len(houses)))

ds_meter = []
ds_appliance = []
ds_status = []
for i in houses:
    ds = pd.read_feather('../data/ukdale/feather_files/UKDALE_%d_train.feather' %(i))
    ds.set_index('datetime', inplace=True)
  
    meter = ds[METER]
    appliances = ds[APPLIANCE]
    
    status = pd.DataFrame()
    for a in range(len(APPLIANCE)):
        status = pd.concat([status, get_status(ds[APPLIANCE[a]], THRESHOLD[a], MIN_OFF[a], MIN_ON[a])], axis=1)
    
    ds_meter.append(meter)
    ds_appliance.append(appliances)
    ds_status.append(status)

ds_len = [len(ds_meter[i]) for i in range(3)]

In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from utils.preprocessing import Power

ds_house_total  = [Power(ds_meter[i], ds_appliance[i], ds_status[i], 
                         SEQ_LEN, BORDER, MAX_POWER, False) for i in range(3)]
dl_test = DataLoader(dataset = ds_house_total[1], batch_size = 32, shuffle=False)

dl_house_test = [DataLoader(dataset = ds_house_total[1], batch_size = 32, shuffle=False)]
dl_house_total = [DataLoader(dataset = ds_house_total[i], batch_size = 32, shuffle=False) for i in range(3)]

for k in range(6):
    globals()["ds_house_filter_" + str(THRESHOLDS[k])] = [Power(datafilter[k][j], ds_appliance[j], ds_status[j], 
                        SEQ_LEN, BORDER, MAX_POWER, False) for j in range(3)]

for l in range(6):
    globals()["dl_house_filter_" + str(THRESHOLDS[l])] = [DataLoader(dataset = globals()["ds_house_filter_" + str(THRESHOLDS[l])][j], batch_size = 32, shuffle=False) for j in range(3)]

In [8]:
from utils.net import PTPNet
from utils.evaluation import print_metrics

###### print no filtered data metrics ######
print_metrics(model = PTPNet(1,3,32).cuda(),
              weights_path = '../data/ukdale/network_weights/UKDALE_network_weights.pth',
              x_test = dl_house_total[1],
              min_off = MIN_OFF,
              min_on = MIN_ON,
              max_power = MAX_POWER,
              appliance_list = APPLIANCE,
              y_test_app = ds_appliance[1],
              y_test_stat = ds_status[1],
              border = BORDER,
              thr = 0.5)
###### print filtered data metrics #######
for i in range(6):
    print('-----------------------------')
    print('Threshold = ' + str(THRESHOLDS[i]))
    print_metrics(model = PTPNet(1,3,32).cuda(),
              weights_path = '../data/ukdale/network_weights/UKDALE_network_weights.pth',
              x_test = globals()["dl_house_filter_" + str(THRESHOLDS[i])][1],
              min_off = MIN_OFF,
              min_on = MIN_ON,
              max_power = MAX_POWER,
              appliance_list = APPLIANCE,
              y_test_app = ds_appliance[1],
              y_test_stat = ds_status[1],
              border = BORDER, thr = 0.5)


fridge
F1 score  : 0.866
Precision : 0.890
Recall    : 0.842
Accuracy  : 0.901
MCC       : 0.788
MAE       : 17.35
SAE       : -0.054

dish_washer
F1 score  : 0.832
Precision : 0.817
Recall    : 0.848
Accuracy  : 0.991
MCC       : 0.828
MAE       : 30.95
SAE       : 0.038

washing_machine
F1 score  : 0.833
Precision : 0.762
Recall    : 0.919
Accuracy  : 0.995
MCC       : 0.835
MAE       : 9.57
SAE       : 0.207
-----------------------------
Threshold = 50

fridge
F1 score  : 0.848
Precision : 0.870
Recall    : 0.827
Accuracy  : 0.888
MCC       : 0.761
MAE       : 18.36
SAE       : -0.049

dish_washer
F1 score  : 0.840
Precision : 0.814
Recall    : 0.867
Accuracy  : 0.991
MCC       : 0.836
MAE       : 30.89
SAE       : 0.065

washing_machine
F1 score  : 0.824
Precision : 0.781
Recall    : 0.871
Accuracy  : 0.995
MCC       : 0.822
MAE       : 9.24
SAE       : 0.116
-----------------------------
Threshold = 100

fridge
F1 score  : 0.700
Precision : 0.697
Recall    : 0.704
Accuracy  : 0.7